In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')  # enable import from src/

In [11]:
import numpy as np
import torch

from src.data import oracle, load_instance

In [15]:
instance = load_instance('../data/raw/97_9.jl')

model = oracle(list(range(instance['jobs'][0])), instance)

A = model.getA().toarray()
b = np.array(model.getAttr('rhs'))
c = np.array(model.getAttr('obj'))

In [89]:
J = instance['jobs'][0]
T = instance['tamanho'][0]

# candidate solution (5 is batch size)
X = torch.ones((5, J*T))
X = X.view(-1, J, T)

n_batch = X.shape[0]

# parameters
soc_inicial = 0.7
limite_inferior = 0.0
ef = 0.9
v_bat = 3.6
q = 5
bat_usage = 5

recurso_p = torch.Tensor(instance['recurso_p'])[:T]
priority = instance['priority'] # prioridade de cada tarefa
uso_p = torch.Tensor(instance['uso_p']) # recurso utilizado por cada tarefa
min_statup = instance['min_statup'] # tempo mínimo de vezes que uma tarefa pode iniciar
max_statup = instance['max_statup'] # tempo máximo de vezes que uma tarefa pode iniciar
min_cpu_time = instance['min_cpu_time'] # tempo mínimo de unidades de tempo que uma tarefa pode consumir em sequência
max_cpu_time = instance['max_cpu_time'] # tempo máximo de unidades de tempo que uma tarefa pode consumir em sequência
min_periodo_job = instance['min_periodo_job'] # tempo mínimo que uma tarefa deve esperar para se repetir
max_periodo_job = instance['max_periodo_job'] # tempo máximo que uma tarefa pode esperar para se repetir
win_min = instance['win_min']
win_max = instance['win_max']

consumo = torch.bmm(uso_p.unsqueeze(0).repeat(n_batch,1,1), X).squeeze(1)
recurso_total = recurso_p + bat_usage * v_bat

bat = recurso_p - consumo
i = bat / v_bat

soc = torch.zeros_like(consumo)
soc[:,0] = 0.7
for t in range(1,T):
    soc[:,t] = soc[:,t-1] + (ef / q) * (i[:,t] / 60)

g = torch.hstack((consumo - recurso_total, soc - 1, limite_inferior - soc))
g.shape  # (batch_size, constraints)

torch.Size([5, 291])